In [1]:
from main import *
import numpy as np
import sys

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def genpairs(n, m, N):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    L.add((i,j))
    while len(L) < N:
        while (i, j) in L:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L
def genpairsnotinL(n,m, N, LL):
    L=set()
    i, j = np.random.randint(0, n), np.random.randint(0, m)
    if (i,j) not in LL:
        L.add((i,j))
    while len(L) < N:
        while (i, j) in L or (i,j) in LL:
            i, j = np.random.randint(0, n), np.random.randint(0,m)
        L.add((i,j))
    return L

def IsAutonomous(input_matrix, output_matrix, aut_species, aut_reactions):
    
    autonomous_species=True
    autonomous_reactions=True
    #n, m =input_matrix.shape
    for s in aut_species:
        if sum(input_matrix[s,r] for r in aut_reactions)<0.5 or sum(output_matrix[s,r] for r in aut_reactions)<0.5:
            autonomous_species=False
    for r in aut_reactions:
        if sum(input_matrix[s,r] for s in aut_species)<0.5 or sum(output_matrix[s,r] for s in aut_species)<0.5:
            autonomous_reactions=False

    return autonomous_species, autonomous_reactions


## Data

In [3]:
# NN=[10, 20, 30, 40, 50]
# MM=[10, 20, 30, 40, 50]
# UB=100
# D=[3,4,5]
# for n in NN:
#     for m in MM:
#         for d in D:
#             for rep in [1,2,3,4,5]:
#                 matrix = np.random.randint(0,UB, size=(n,m))
#                 pairs1=genpairs(n, m, n*m/d)
#                 pairs2=genpairsnotinL(n, m, n*m/d, pairs1)

#                 output_matrix = np.zeros((n,m))
#                 input_matrix = np.zeros((n,m))
#                 for i in range(n):
#                     for j in range(m):
#                         if (i,j) in pairs1:
#                             input_matrix[i,j]=matrix[i,j]
#                         elif (i,j) in pairs2:
#                             output_matrix[i,j]=matrix[i,j]

#                 np.savetxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep), input_matrix, fmt="%d")
#                 np.savetxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep), output_matrix, fmt="%d")

## Growth factor

In [4]:
import pandas as pd
np.set_printoptions(suppress=True)
print("n \t m \t d \t rep \t steps \t steps_a \t time \t time_a \t alpha \t alpha_a \t |a|")
df = pd.DataFrame(columns=['n', 'm', 'd', 'rep', 'steps', 'time', 'alpha', 'x', 'IsAut_Sp', 'IsAut_Re', 'steps', 'time_a', 'alpha_a', 'x_a', '|a|', 'a', 'IsAut_Sp_a', 'IsAut_Re_a'])
row=0
NN=[10, 20, 30, 40, 50]
MM=[10, 20, 30, 40, 50]
for n in NN:
    for m in MM:
        D=[3,4,5]
        for d in D:
            for rep in [1,2,3,4,5]:
                input_matrix= np.loadtxt("instances/left_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                output_matrix= np.loadtxt("instances/right_n%dm%dd%d_%d.txt"%(n,m,d,rep))
                #XX = code.growthRateGraph(output_matrix, input_matrix, 10000)
                XX = code.growthRateGraph(output_matrix, input_matrix, 10000)
                xsol = XX[0]
                alphasol =XX[1]
                step = XX[2]
                time=XX[4]
                XX_a = code.growthRateGraph_a(output_matrix, input_matrix, 100000)
                xsol_a = XX_a[0]
                alphasol_a =XX_a[1]
                step_a = XX_a[2]
                time_a=XX_a[4]
                asol=XX_a[5]
                # print("steps: ", step)
                # print("x: ", xsol)
                # print("alpha: ", alphasol)
                # print("time: ", time)
                isA_sp, isA_re=IsAutonomous(input_matrix, output_matrix, range(n), range(m))
                isA_sp_a, isA_re_a=IsAutonomous(input_matrix, output_matrix, asol, range(m))
                isA=False
                if isA_sp and isA_re:
                    isA=True
                isA_a=False
                if isA_sp_a and isA_re_a:
                    isA_a=True
                print("%d \t %d \t %d \t %d \t %d \t %d \t %.4f \t %.4f \t %.4f \t %.4f \t %d"%(n,m,d,rep,step,step_a, time, time_a, alphasol, alphasol_a, len(asol)), isA, isA_a)
                df.loc[row] = [n,m,d,rep,step,time,alphasol,xsol,  isA_sp, isA_re, step_a,time_a,alphasol_a,xsol_a,len(asol), asol, isA_sp_a, isA_re_a]
                row+=1
df.to_csv('GrowthRate_Experiments.csv', index=False)  

n 	 m 	 d 	 rep 	 steps 	 steps_a 	 time 	 time_a 	 alpha 	 alpha_a 	 |a|
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-19
10 	 10 	 3 	 1 	 25 	 25 	 0.1322 	 0.3186 	 0.9611 	 0.9611 	 10 True True
10 	 10 	 3 	 2 	 37 	 37 	 0.1736 	 0.7805 	 0.6229 	 0.6229 	 10 True True
10 	 10 	 3 	 3 	 32 	 32 	 0.1529 	 0.3748 	 0.8187 	 0.8187 	 10 True True
10 	 10 	 3 	 4 	 31 	 31 	 0.1494 	 0.5210 	 0.4282 	 0.4282 	 10 True True
10 	 10 	 3 	 5 	 0 	 0 	 0.0046 	 0.0058 	 -1.0000 	 0.1691 	 9 False True
10 	 10 	 4 	 1 	 0 	 0 	 0.0047 	 0.0049 	 0.0000 	 -1.0000 	 0 False False
10 	 10 	 4 	 2 	 27 	 26 	 0.1322 	 0.3754 	 0.7915 	 0.7915 	 10 True True
10 	 10 	 4 	 3 	 0 	 0 	 0.0045 	 0.0047 	 -1.0000 	 -1.0000 	 0 False False
10 	 10 	 4 	 4 	 0 	 0 	 0.0045 	 0.0050 	 -1.0000 	 -1.0000 	 0 False False
10 	 10 	 4 	 5 	 0 	 0 	 0.0046 	 0.0048 	 -1.0000 	 -1.0000 	 0 False False
10 	 10 	 5 	 1 	 0 	 0 	 0.0044 	 0.0047 	 -1.0000 	 -1.0000 	

In [5]:
import pandas as pd
SM0 = pd.DataFrame(SM)
SM0.columns = ['']*SM0.shape[1]
print(SM0.to_string(index=False))

I0 = pd.DataFrame(input_matrix_sol)
I0.columns = ['']*I0.shape[1]
print(I0.to_string(index=False))

O0 = pd.DataFrame(output_matrix_sol)
O0.columns = ['']*O0.shape[1]
print(O0.to_string(index=False))

SMsol0 = pd.DataFrame(SMsol)
SMsol0.columns = ['']*SMsol0.shape[1]
print(SMsol0.to_string(index=False))

NameError: name 'SM' is not defined

In [15]:
alpha_hand =  np.min([sum(output_matrix[s, r] * xsol[r] 
                         for r in zsol)
                     /
                     sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol) 
                     for s in asol if sum(input_matrix[s, r] * xsol[r] 
                         for r in zsol)>0.0001])
alpha_hand

15.684386344640638